<a href="https://colab.research.google.com/github/kentyanron/ai_app_2022-12-8/blob/main/AI%E3%83%97%E3%83%AD%E3%82%B0%E3%83%A9%E3%83%9F%E3%83%B3%E3%82%B0_%E8%AA%B2%E9%A1%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

RNNによる自動作曲をするアプリのプログラムです。アプリ化がうまくいかないため、アプリ化に関わるプログラムは省略し、自動作曲をする部分のみしております。基本的に授業資料（11_polyphony_rnn.ipynb）をベースとし、Performance RNN（高度なピアノ演奏の作曲ができるライブラリ）を用いております。また、起点となるメロディの設定の際には、pitchをランダムな値になるようにすることで、毎回違った雰囲気の曲になるようにしました。

In [1]:
!apt-get update -qq && apt-get install -qq libfluidsynth1 fluid-soundfont-gm build-essential libasound2-dev libjack-dev
!pip install -qU pyfluidsynth pretty_midi
!pip install -qU magenta

Selecting previously unselected package fluid-soundfont-gm.
(Reading database ... 124016 files and directories currently installed.)
Preparing to unpack .../fluid-soundfont-gm_3.1-5.1_all.deb ...
Unpacking fluid-soundfont-gm (3.1-5.1) ...
Selecting previously unselected package libfluidsynth1:amd64.
Preparing to unpack .../libfluidsynth1_1.1.9-1_amd64.deb ...
Unpacking libfluidsynth1:amd64 (1.1.9-1) ...
Setting up fluid-soundfont-gm (3.1-5.1) ...
Setting up libfluidsynth1:amd64 (1.1.9-1) ...
Processing triggers for libc-bin (2.27-3ubuntu1.6) ...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 40.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.1/51.1 KB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 62.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.8/69.8 KB 6.3 MB/s eta 0:00:00
     ━━━

In [6]:
import magenta
import note_seq
from note_seq.protobuf import music_pb2

seed = music_pb2.NoteSequence()  # NoteSequence

import random
a = random.randint(30,90)
b = random.randint(30,90)
c = random.randint(30,90)
d = random.randint(30,90)
e = random.randint(30,90)
f = random.randint(30,90)
g = random.randint(30,90)

# notesにnoteを追加
seed.notes.add(pitch=a, start_time=0.0, end_time=0.3, velocity=80)
seed.notes.add(pitch=b, start_time=0.3, end_time=0.6, velocity=80)
seed.notes.add(pitch=c, start_time=0.6, end_time=0.9, velocity=80)
seed.notes.add(pitch=d, start_time=0.9, end_time=1.2, velocity=80)
seed.notes.add(pitch=e, start_time=1.2, end_time=1.5, velocity=80)
seed.notes.add(pitch=f, start_time=1.5, end_time=1.8, velocity=80)
seed.notes.add(pitch=g, start_time=1.8, end_time=2.1, velocity=80)

seed.total_time = 2.1  # 所要時間
seed.tempos.add(qpm=132);  # 曲のテンポを指定

note_seq.plot_sequence(seed)  # NoteSequenceの可視化
note_seq.play_sequence(seed, synth=note_seq.fluidsynth)  # NoteSequenceの再生

In [3]:
from magenta.models.performance_rnn import performance_sequence_generator
from magenta.models.shared import sequence_generator_bundle

# モデルの初期化
note_seq.notebook_utils.download_bundle("performance_with_dynamics.mag", "/models/")  # Bundle（.magファイル）をダウンロード
bundle = sequence_generator_bundle.read_bundle_file("/models/performance_with_dynamics.mag")  # Bundleの読み込み
generator_map = performance_sequence_generator.get_generator_map()
performance_rnn = generator_map["performance_with_dynamics"](checkpoint=None, bundle=bundle)  # 生成器の設定
performance_rnn.initialize()  # 初期化

'model_variables' collection should be of type 'byte_list', but instead is of type 'node_list'.


In [8]:
from note_seq.protobuf import generator_pb2

total_time = 90 # 曲の長さ（秒）
temperature = 1 # 曲の「ランダム度合い」を決める定数

base_end_time = max(note.end_time for note in seed.notes)  #ベース曲の終了時刻

# 生成器に関する設定
generator_options = generator_pb2.GeneratorOptions()  # 生成器のオプション
generator_options.args["temperature"].float_value = temperature  # ランダム度合い
generator_options.generate_sections.add(
    start_time=base_end_time,  # 作曲開始時刻
    end_time=total_time)  # 作曲終了時刻

# 曲の生成
gen_seq = performance_rnn.generate(seed, generator_options)

note_seq.plot_sequence(gen_seq)  # NoteSequenceの可視化
note_seq.play_sequence(gen_seq, synth=note_seq.fluidsynth)  # NoteSequenceの再生

In [5]:
from google.colab import files

note_seq.sequence_proto_to_midi_file(gen_seq, "performance_rnn.mid")  #MIDI　データに変換し保存
files.download("performance_rnn.mid")  # ダウンロード

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>